# Rule-based initial filtering

1. Minimum experience requirements: must have minimum years of experience
2. Minimum skill requirements: must match at least one required skill
3. Minimum education requirements: must have at least one required degree

In [4]:
import json

In [5]:
with open("../output/parsed_resume_info.json", "r") as file:
    resume_info = json.load(file)

In [6]:
resume_info

{'Education': [{'Name': 'Texas A&M University',
   'Majors': ['BS in Computer Science'],
   'Minors': ['Statistics', 'Math'],
   'Graduation Year': 2026,
   'GPA': 4.0}],
 'Work Experience': {'Experiences': [{'Company': 'Full Stack Developer Intern',
    'Role': 'Full Stack Developer Intern',
    'Contributions': ['Built CRM system with ASP.NET MVC serving 10 users, improving employee efficiency by 50%',
     'Optimized MySQL database performance by eliminating 1,000 duplicate records, improving query speed by 10%',
     'Designed scalable SQL database architecture to support complex entity relationships',
     'Created role-based authorization system with Entity Framework to facilitate project management for managers']},
   {'Company': 'Software Engineering Intern',
    'Role': 'Software Engineering Intern',
    'Contributions': ['Redesigned 30 year old Java data analysis suite architecture, cutting developer onboarding time by 3 weeks',
     'Used MATLAB profiler to find bottleneck i

In [7]:
with open("../output/parsed_job_desc.json", "r") as file:
    job_info = json.load(file)

In [8]:
job_info

{'Required Skills': ['Python',
  'C',
  'C++',
  'Linux kernel',
  'application-level system software'],
 'Preferred Skills': ['Deep Learning',
  'Transformer',
  'NLP',
  'LLM technology',
  'QNX RTOS',
  'CAN and tools',
  'RADAR',
  'LiDAR'],
 'Required Experience': ['Experience in research and development of Deep Learning/Transformer/NLP/LLM technology',
  'Strong Linux kernel, application-level system software experience'],
 'Preferred Experience': ['Prior experience in the Automotive field',
  'Background in QNX RTOS and debug tools',
  'Experience in developing and maintaining AI or machine learning infrastructure, preferably in the context of large language models'],
 'Required Education': ['Pursuing BS or MS in CS/CE/EE or equivalent program'],
 'Preferred Education': []}

### Filter minimum education

May be more difficult to filter this with rules (how do you match "Pursuing BS or MS in CS/CE/EE or equivalent" to resumes?)

For now, just toss out resumes without any education information

In [9]:
def meets_min_education(resume_info: dict) -> bool:
    return len(resume_info["Education"]) > 0

In [10]:
meets_min_education(resume_info)

True

### Filter minimum skill requirements

Does the resume contain *at least one* of the required skills?

In [11]:
def meets_min_skills(job_info: dict, resume_info: dict) -> bool:
    job_skills = job_info["Required Skills"]
    if len(job_skills) == 0:
        return True
    
    resume_skills = resume_info["Skills"]["Technical Skills"]
    for skill in job_skills:
        if skill in resume_skills:
            return True
    return False

In [12]:
meets_min_skills(job_info, resume_info)

True

### Filter minimum experience 

Maybe use fuzzy matching here to check whether any of the resume's experiences are similar to the job's required experiences

In [13]:
from rapidfuzz import fuzz

In [33]:
def meets_min_experience(job_info: dict, resume_info: dict) -> bool:
    job_exp = job_info["Required Experience"]
    if len(job_exp) == 0:
        return True
    
    resume_skills = resume_info["Skills"]["Technical Skills"] + resume_info["Skills"]["Domain-Specific Skills"]
    for experience in job_exp:
        for skill in resume_skills:
            if fuzz.partial_ratio(skill, experience) > 45:
                print(skill, ":", experience, fuzz.partial_ratio(skill, experience))

In [34]:
meets_min_experience(job_info, resume_info)

Python : Experience in research and development of Deep Learning/Transformer/NLP/LLM technology 50.0
Bash : Experience in research and development of Deep Learning/Transformer/NLP/LLM technology 50.0
React : Experience in research and development of Deep Learning/Transformer/NLP/LLM technology 60.0
Spring Boot : Experience in research and development of Deep Learning/Transformer/NLP/LLM technology 45.45454545454546
Pandas : Experience in research and development of Deep Learning/Transformer/NLP/LLM technology 50.0
LangChain : Experience in research and development of Deep Learning/Transformer/NLP/LLM technology 55.55555555555556
OpenAI : Experience in research and development of Deep Learning/Transformer/NLP/LLM technology 50.0
TensorFlow : Experience in research and development of Deep Learning/Transformer/NLP/LLM technology 52.63157894736843
Anaconda : Experience in research and development of Deep Learning/Transformer/NLP/LLM technology 50.0
Jupyter : Experience in research and deve

There doesn't seem to be an easy to way create a rule-based system for filtering out minimum experience requirements; this likely requires a little bit of reasoning